In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [6]:
ratings = pd.read_csv("drive/MyDrive/Dataset2/General Result/ratings.csv")
movies = pd.read_csv("drive/MyDrive/Dataset2/General Result/Movies.csv")

In [7]:
movies.head()

,movieId,title,genres
0,1096197,No Way Up,"Action,Horror,Thriller"
1,823464,Godzilla x Kong: The New Empire,"Action,Science Fiction,Adventure,Fantasy"
2,934632,Rebel Moon — Part Two: The Scargiver,"Science Fiction,Action,Drama,Adventure"
3,1011985,Kung Fu Panda 4,"Animation,Action,Family,Comedy,Fantasy"
4,693134,Dune: Part Two,"Science Fiction,Adventure"


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
movies.shape

(8057, 3)

In [11]:
ratings.shape

(100836, 3)

In [12]:
x=ratings['userId'].value_counts()>200

In [13]:
y = x[x].index

In [14]:
y

Index([414, 599, 474, 448, 274, 610,  68, 380, 606, 288,
       ...
       119, 563, 263,  73, 586, 220, 246, 234, 452, 385],
      dtype='int64', name='userId', length=133)

In [15]:
ratings = ratings[ratings['userId'].isin(y)]

In [16]:
ratings.shape

(68284, 3)

In [17]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [18]:
movies_ratings = pd.merge(ratings, movies, on="movieId")
movies_ratings.head()

,userId,movieId,rating,title,genres
0,1,3,4.0,Shadows in Paradise,"Drama,Comedy,Romance"
1,6,3,5.0,Shadows in Paradise,"Drama,Comedy,Romance"
2,19,3,3.0,Shadows in Paradise,"Drama,Comedy,Romance"
3,42,3,4.0,Shadows in Paradise,"Drama,Comedy,Romance"
4,51,3,4.0,Shadows in Paradise,"Drama,Comedy,Romance"


In [19]:
movies_ratings.shape

(10317, 5)

In [20]:
numbrer_rating = movies_ratings.groupby('title')['rating'].count().reset_index()

In [21]:
numbrer_rating.rename(columns={'rating':'No.Of Rating'}, inplace=True)

In [22]:
numbrer_rating.head()

,title,No.Of Rating
0,10 Things I Hate About You,7
1,"10,000 BC",3
2,15 Minutes,1
3,16 Blocks,1
4,"20,000 Leagues Under the Sea",29


In [43]:
final_rating = movies_ratings.merge(numbrer_rating, on='title')

In [44]:
final_rating.head()

,userId,movieId,rating,title,genres,No.Of Rating
0,1,3,4.0,Shadows in Paradise,"Drama,Comedy,Romance",25
1,6,3,5.0,Shadows in Paradise,"Drama,Comedy,Romance",25
2,19,3,3.0,Shadows in Paradise,"Drama,Comedy,Romance",25
3,42,3,4.0,Shadows in Paradise,"Drama,Comedy,Romance",25
4,51,3,4.0,Shadows in Paradise,"Drama,Comedy,Romance",25


In [45]:
final_rating.shape

(10317, 6)

In [46]:
final_rating = final_rating[final_rating['No.Of Rating'] >= 10]

In [47]:
final_rating.shape

(8422, 6)

In [48]:
movie_pivot = final_rating.pivot_table(columns='userId', index='title', values='rating')

In [49]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
movie_pivot.head()

userId,1,4,6,18,19,20,21,28,41,42,45,50,51,57,62,63,64,66,68,73,82,84,89,91,103,104,105,111,119,122,125,132,135,140,156,160,169,177,182,186,187,198,199,200,202,212,217,219,220,221,222,226,232,234,239,246,249,263,274,275,282,288,290,292,294,298,304,305,307,312,313,318,325,328,330,332,339,352,354,356,357,365,368,372,380,381,382,385,387,391,414,425,428,432,434,438,448,452,453,462,469,474,477,480,483,484,489,495,509,514,517,525,534,555,560,561,562,563,567,573,580,586,590,594,596,597,599,600,603,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"20,000 Leagues Under the Sea",NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.5,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,0.5,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.5,NaN,NaN,NaN,NaN,3.0,NaN
2001: A Space Odyssey,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,2.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.5,3.0,NaN,NaN,3.0,NaN,4.5,NaN,3.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,2.5,NaN,3.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,1.0,NaN,NaN,NaN,NaN
25th Hour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,2.0,3.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.5
28 Days Later,NaN,NaN,3.0,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,2.5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.5,NaN
28 Weeks Later,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.5,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,1.5,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.5,NaN,NaN,NaN,1.5,NaN,2.0,NaN,NaN,2.5,NaN,2.5,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.5,NaN,2.0,NaN,NaN,NaN,NaN,2.0,1.5,3.0,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,2.0,NaN,2.0,1.0,1.5,NaN,NaN,3.0,NaN,2.0,NaN


In [50]:
movie_pivot.shape

(272, 133)

In [51]:
movie_pivot.fillna(0, inplace=True)

In [52]:
movie_pivot.head()

userId,1,4,6,18,19,20,21,28,41,42,45,50,51,57,62,63,64,66,68,73,82,84,89,91,103,104,105,111,119,122,125,132,135,140,156,160,169,177,182,186,187,198,199,200,202,212,217,219,220,221,222,226,232,234,239,246,249,263,274,275,282,288,290,292,294,298,304,305,307,312,313,318,325,328,330,332,339,352,354,356,357,365,368,372,380,381,382,385,387,391,414,425,428,432,434,438,448,452,453,462,469,474,477,480,483,484,489,495,509,514,517,525,534,555,560,561,562,563,567,573,580,586,590,594,596,597,599,600,603,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"20,000 Leagues Under the Sea",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.5,0.0,0.0,0.0,0.0,3.0,0.0
2001: A Space Odyssey,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.5,3.0,0.0,0.0,3.0,0.0,4.5,0.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,1.0,0.0,0.0,0.0,0.0
25th Hour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.5
28 Days Later,0.0,0.0,3.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.5,0.0
28 Weeks Later,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,1.5,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.5,0.0,0.0,0.0,1.5,0.0,2.0,0.0,0.0,2.5,0.0,2.5,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.5,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.5,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,2.0,0.0,2.0,1.0,1.5,0.0,0.0,3.0,0.0,2.0,0.0


In [54]:
movie_sparse = csr_matrix(movie_pivot)
movie_sparse

<272x133 sparse matrix of type '<class 'numpy.float64'>'
	with 8389 stored elements in Compressed Sparse Row format>

In [55]:
model = NearestNeighbors(algorithm='brute')
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute')

In [56]:
np.where(movie_pivot.index == 200)

(array([], dtype=int64),)

In [57]:
def based_content_filtering(movie_id):
    distances, suggestions = model.kneighbors(movie_pivot.iloc[movie_id, :].values.reshape(1, -1), n_neighbors=11)
    suggested_movie_ids = []
    for i in range(len(suggestions[0])):
        if i != 0:
            suggested_movie_ids.append(suggestions[0][i])
    return suggested_movie_ids

In [58]:
id = int(input('Enter the movie id : '))
movie_pivot.index[id]
suggestions_movie = based_content_filtering(id)
suggestions_movie

Enter the movie id : 50


[177, 245, 152, 248, 206, 188, 219, 216, 99, 223]